PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras as k
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import adam_v2
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split,KFold,GridSearchCV

In [2]:
df = pd.read_csv("forestfires.csv")
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,0,1,0,0,0,0,small
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,0,1,0,small
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,0,1,0,small
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,0,1,0,0,0,0,small
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,0,1,0,0,0,0,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,0,large
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,0,large
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,0,large
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,0,small


In [3]:
df.size_category.value_counts()

small    378
large    139
Name: size_category, dtype: int64

In [4]:
lb = LabelEncoder()
df['size_category_label']=lb.fit_transform(df.size_category)
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category,size_category_label
0,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,small,1
1,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,1,0,small,1
2,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,1,0,small,1
3,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,small,1
4,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,1,0,0,0,0,small,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,large,0
513,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,large,0
514,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,large,0
515,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,small,1


In [5]:
df.month.replace(('jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'),(1,2,3,4,5,6,7,8,9,10,11,12), inplace=True)
df.day.replace(('mon','tue','wed','thu','fri','sat','sun'),(1,2,3,4,5,6,7), inplace=True)
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,...,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep,size_category,size_category_label
0,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,...,0,0,0,1,0,0,0,0,small,1
1,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,...,0,0,0,0,0,0,1,0,small,1
2,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,...,0,0,0,0,0,0,1,0,small,1
3,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,...,0,0,0,1,0,0,0,0,small,1
4,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,...,0,0,0,1,0,0,0,0,small,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,...,0,0,0,0,0,0,0,0,large,0
513,8,7,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,...,0,0,0,0,0,0,0,0,large,0
514,8,7,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,...,0,0,0,0,0,0,0,0,large,0
515,8,6,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,...,0,0,0,0,0,0,0,0,small,1


In [6]:
df.drop(df.iloc[:, 11:31],axis=1,inplace=True)
df

,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,size_category_label
0,3,5,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00,1
1,10,2,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00,1
2,10,6,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00,1
3,3,5,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00,1
4,3,7,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...
512,8,7,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44,0
513,8,7,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29,0
514,8,7,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16,0
515,8,6,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00,1


In [7]:
print("Statistical Description:", df.describe())

Statistical Description:             month         day        FFMC         DMC          DC         ISI  \
count  517.000000  517.000000  517.000000  517.000000  517.000000  517.000000   
mean     7.475822    4.259188   90.644681  110.872340  547.940039    9.021663   
std      2.275990    2.072929    5.520111   64.046482  248.066192    4.559477   
min      1.000000    1.000000   18.700000    1.100000    7.900000    0.000000   
25%      7.000000    2.000000   90.200000   68.600000  437.700000    6.500000   
50%      8.000000    5.000000   91.600000  108.300000  664.200000    8.400000   
75%      9.000000    6.000000   92.900000  142.400000  713.900000   10.800000   
max     12.000000    7.000000   96.200000  291.300000  860.600000   56.100000   

             temp          RH        wind        rain         area  \
count  517.000000  517.000000  517.000000  517.000000   517.000000   
mean    18.889168   44.288201    4.017602    0.021663    12.847292   
std      5.806625   16.317469    1.

In [8]:
print("Shape:", df.shape)
print("\n Data Types:", df.dtypes)
print("\n Correlation:", df.corr(method='pearson'))

Shape: (517, 12)

 Data Types: month                    int64
day                      int64
FFMC                   float64
DMC                    float64
DC                     float64
ISI                    float64
temp                   float64
RH                       int64
wind                   float64
rain                   float64
area                   float64
size_category_label      int32
dtype: object

 Correlation:                         month       day      FFMC       DMC        DC  \
month                1.000000 -0.050837  0.291477  0.466645  0.868698   
day                 -0.050837  1.000000 -0.041068  0.062870  0.000105   
FFMC                 0.291477 -0.041068  1.000000  0.382619  0.330512   
DMC                  0.466645  0.062870  0.382619  1.000000  0.682192   
DC                   0.868698  0.000105  0.330512  0.682192  1.000000   
ISI                  0.186597  0.032909  0.531805  0.305128  0.229154   
temp                 0.368842  0.052190  0.431532  0.4695

In [40]:
X = df.iloc[:,0:11]
#df2 = df.iloc[:,11]
#X = pd.concat((df1,df2),axis=1,join='inner')
Y = df.iloc[:,-1]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [44]:
std_X_train = StandardScaler().fit_transform(X_train)
std_X_tst = StandardScaler().fit_transform(X_test)

In [45]:
pd.DataFrame(std_X_train).describe()

,0,1,2,3,4,5,6,7,8,9,10
count,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02,3.610000e+02
mean,-1.350105e-16,1.070243e-16,-9.472263e-17,-3.450226e-17,5.843279e-18,1.476197e-17,-3.017746e-16,1.476197e-16,1.730014e-16,3.006213e-17,-6.243082e-17
std,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00,1.001388e+00
min,-2.838113e+00,-1.569929e+00,-1.215512e+01,-1.682969e+00,-2.150181e+00,-2.240186e+00,-2.893261e+00,-1.798694e+00,-1.723691e+00,-7.097384e-02,-2.460602e-01
25%,-1.986314e-01,-1.089637e+00,-5.988692e-02,-6.262360e-01,-4.601902e-01,-5.760742e-01,-5.839526e-01,-7.102311e-01,-7.119896e-01,-7.097384e-02,-2.460602e-01
50%,2.412823e-01,3.512384e-01,1.769429e-01,-5.617612e-02,4.725219e-01,-8.964148e-02,4.112342e-02,-1.659998e-01,1.868331e-02,-7.097384e-02,-2.349213e-01
75%,6.811959e-01,8.315303e-01,3.968563e-01,4.426263e-01,6.727313e-01,4.735964e-01,6.488362e-01,5.596420e-01,8.055619e-01,-7.097384e-02,-1.008420e-01
max,2.000937e+00,1.311822e+00,9.550979e-01,2.716802e+00,1.257375e+00,3.571405e+00,2.506701e+00,3.341269e+00,3.053786e+00,1.856288e+01,1.514789e+01


In [47]:
def create_model(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 11,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_squared_error',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0)

# Define the grid search parameters

batch_size = [20,40]
epochs = [200,300]
learning_rate = [0.01,0.001]
dropout_rate = [0.1,0.2]
activation_function = ['relu','tanh','linear']
init = ['uniform','normal']
neuron1 = [40,20]
neuron2 = [30,60]

# Make a dictionary of the grid search parameters

param_grids = dict(batch_size = batch_size,epochs = epochs,learning_rate = learning_rate,dropout_rate = dropout_rate,
                   activation_function = activation_function,init = init,neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(std_X_train,y_train)

# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

<ipython-input-47-30a172ddbd2e>:15: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasRegressor(build_fn = create_model,verbose = 0)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=200, init=uniform, learning_rate=0.01, neuron1=40, neuron2=30 
[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=200, init=uniform, learning_rate=0.01, neuron1=40, neuron2=30, score=-0.037, total=  10.9s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=200, init=uniform, learning_rate=0.01, neuron1=40, neuron2=30 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.8s remaining:    0.0s


[CV]  activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=200, init=uniform, learning_rate=0.01, neuron1=40, neuron2=30, score=-0.046, total=  11.7s
[CV] activation_function=relu, batch_size=20, dropout_rate=0.1, epochs=200, init=uniform, learning_rate=0.01, neuron1=40, neuron2=30 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   22.5s remaining:    0.0s


...

[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=20, neuron2=30, score=-0.024, total=   3.4s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=20, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=20, neuron2=30, score=-0.000, total=   3.8s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=20, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=20, neuron2=30, score=-0.015, total=   4.3s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, learning_rate=0.01, neuron1=20, neuron2=60 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=uniform, lear

[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=40, neuron2=30, score=-0.022, total=   3.9s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=40, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=40, neuron2=30, score=-0.024, total=   3.4s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=40, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=40, neuron2=30, score=-0.021, total=   3.3s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.01, neuron1=40, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_ra

[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.001, neuron1=40, neuron2=60, score=-0.000, total=   4.1s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.001, neuron1=40, neuron2=60 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.001, neuron1=40, neuron2=60, score=-0.013, total=   3.7s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.001, neuron1=20, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.001, neuron1=20, neuron2=30, score=-0.004, total=   3.4s
[CV] activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learning_rate=0.001, neuron1=20, neuron2=30 
[CV]  activation_function=linear, batch_size=40, dropout_rate=0.2, epochs=300, init=normal, learn

[Parallel(n_jobs=1)]: Done 1920 out of 1920 | elapsed: 171.1min finished


Best : -0.008927286569087301, using {'activation_function': 'tanh', 'batch_size': 20, 'dropout_rate': 0.1, 'epochs': 300, 'init': 'uniform', 'learning_rate': 0.001, 'neuron1': 40, 'neuron2': 60}


In [ ]:
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

In [139]:
def model_Creation(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 11,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = adam_v2.Adam(learning_rate = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model


def Train_Test_Validation(X,Y):
    batch_size = 20
    epochs = 300
    learning_rate = 0.001
    dropout_rate = 0.1
    activation_function = 'tanh'
    init = 'uniform'
    neuron1 = 40
    neuron2 = 60
    #neuron3 = 20
    final_model = model_Creation(learning_rate,dropout_rate,activation_function,init,neuron1,neuron2)
    #model_classifier = KerasClassifier(build_fn = final_model,verbose = 0,batch_size = batch_size,epochs = epochs)
    final_model.fit(X, Y,verbose = 0,batch_size = batch_size,epochs = epochs)
    scores = final_model.evaluate(X, Y)
    print("%s: %.2f%%" % (final_model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (final_model.metrics_names[0], scores[0]*100))
    return final_model


final_model = Train_Test_Validation(std_X_train, y_train)
#Train_Test_Validation(std_X_tst,y_test)

12/12 [==============================] - 0s 1ms/step - loss: 0.0141 - accuracy: 0.9917
accuracy: 99.17%
loss: 1.41%


In [155]:
#Training Data
#pred_train = first_model.predict(np.array(x_train))
y_predict = final_model.predict(std_X_train)
y_predict = pd.Series([i[0] for i in y_predict])# Printing the metrics o/p is in obj dtype so converting to float so using i[0q]
#y_predict
#size = ["small","large"]
#0 for small and 1 for large

pred_train_class = pd.Series([0]*361)#len(std_X_train) is 361 
pred_train_class[[i>0.5 for i in y_predict]]= 1
pred_train_class.unique()
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
print(accuracy_score(y_train,pred_train_class))
print('\n',classification_report(y_train,pred_train_class))
print('\n',confusion_matrix(y_train,pred_train_class))

0.997229916897507

               precision    recall  f1-score   support

           0       1.00      0.99      0.99        98
           1       1.00      1.00      1.00       263

    accuracy                           1.00       361
   macro avg       1.00      0.99      1.00       361
weighted avg       1.00      1.00      1.00       361


 [[ 97   1]
 [  0 263]]


In [156]:
#Testing data
y_predict_tst = final_model.predict(std_X_tst)
y_predict_tst = pd.Series([i[0] for i in y_predict_tst])# Printing the metrics o/p is in obj dtype so converting to float so using i[0q]
#y_predict
#size = ["small","large"]
#0 for small and 1 for large

pred_train_class_tst = pd.Series([0]*156)#len(std_X_train) is 156 
pred_train_class_tst[[i>0.5 for i in y_predict_tst]]= 1
pred_train_class_tst.unique()
print(accuracy_score(y_test,pred_train_class_tst))
print('\n',classification_report(y_test,pred_train_class_tst))
print('\n',confusion_matrix(y_test,pred_train_class_tst))

0.9487179487179487

               precision    recall  f1-score   support

           0       0.84      1.00      0.91        41
           1       1.00      0.93      0.96       115

    accuracy                           0.95       156
   macro avg       0.92      0.97      0.94       156
weighted avg       0.96      0.95      0.95       156


 [[ 41   0]
 [  8 107]]


In [153]:
len(std_X_tst)

156